In [1]:
import random 
import matplotlib.pyplot as plt

In [2]:
def create_nodes_messages(min_input,max_input,num_nodes):
    """The nodes array keeps track of the value for the node, which at first is their input, 
    which is generated as a random number between the given parameters.
    The messages are kept track in al ist where the index of the node 
     are the messages meant for it. Note that the list will store the messages as [message,delay of message]
    """
    nodes = []
    messages_delay = []
    for i in xrange(num_nodes):
        node_input = random.uniform(min_input,max_input)
#        id_input = (i,node_input)
        nodes.append(node_input)
        messages_delay.append([])
    return nodes,messages_delay

In [11]:
def multicast(node_id,messages_delay,message,min_delay,max_delay):
    """Updates list_message_delay by generating a random delay for each node and
    appending the [message, delay] to the list of messages for the given node
        Also note that a node won't send a message to itself,therefore the function will only send the message if i!=node_id
    """
    for i in xrange(len(messages_delay)):
        if i!=node_id:
            delay = random.randint(min_delay,max_delay)
            messages_delay[i].append([message,delay])
    return messages_delay

In [1]:
def receive_update(messages_delay,nodes,node_id,debug):
    """Updates the given messages for a given node by either receiving the node if delay==0
        or decreasing delay by 1. 
    """
    message_for_node = messages_delay[node_id]
    messages_received = []
    length_messages = len(message_for_node)
    i = 0
    while i<length_messages:
        message_delay = message_for_node[i]
        if(message_delay[1]==0):
            messages_received.append(message_delay[0])
            del(messages_delay[node_id][i])
            i-=1
            length_messages -=1
        else:
            message_delay[1]-=1
        i+=1
    if(len(messages_received) != 0):
        sum_received = sum(messages_received)
        amount = len(messages_received)
        avg_message = sum_received/amount
        if debug: 
            print "Node # {} : Messages Received: {}".format(node_id,messages_received)
            print "Sum messages: {}".format(sum_received) 
            print "Amount of messages: {}".format(amount) 
            print "Avg of messages received {}".format(avg_message)
        nodes[node_id] = (nodes[node_id] + avg_message)/2.0
    
    return nodes, messages_delay


In [14]:
#Evaulates if the nodes have converged
def is_converged(nodes,e):
    for i in range(len(nodes)):
        for j in range(i+1,len(nodes)):
            if(abs(nodes[i]-nodes[j])>e):
                return False
    return True

In [2]:
#Also note that in our definition of multicast a node does not send messages to itself
def simul_concensus_protocol(min_delay,max_delay,max_input = 1,num_nodes=100, e=1,debug=False):
    min_input = 0
    num_runs = 0
    nodes , messages_delay = create_nodes_messages(min_input,max_input,num_nodes)
    while not(is_converged(nodes,e)):
        num_runs +=1
        for i in xrange(num_nodes):
            message = nodes[i]
            if debug:
                print("Generating message {} for all nodes".format(message))
            messages_delay = multicast(i,messages_delay,message,min_delay,max_delay)
        for i in xrange(num_nodes):
            if debug:
                print "Messages for node # {} : {}".format(i,messages_delay[i])
                print "Previous value of  node # {}: {} : ".format(i,nodes[i])
            nodes,messages_delay = receive_update(messages_delay,nodes,i,debug=debug)
            if debug:
                print "Updated value of  node # {} :{} ".format(i,nodes[i])
    if debug:
        print("Final node values {}".format(nodes))
    return num_runs

In [3]:
def plot_protocol(num_nodes,input_range,num_runs =10, min_delay=0,max_delay=5,e=0.1,steps=1, simul_concensus_protocol=simul_concensus_protocol):
    averages = []
    inputs = [i for i in range(0,input_range+1,steps)] #python increases i by the parameter steps for each loop
    for input_ in inputs:
        count = 0.0
        for i in xrange(num_runs):
            count+= simul_concensus_protocol(min_delay,max_delay,max_input=input_,num_nodes=num_nodes,e=e)
        avg = count/num_runs
        averages.append(avg)
    plt.plot(inputs,averages,label="Epsilon : {} max_delay {}".format(e,max_delay))
    plt.xlabel("Max_inputs (min_input =0)")
    plt.ylabel("Average rounds before convergence for {} runs".format(num_runs))
    plt.title("Simulation for Concensus Protocol with {} nodes ".format(num_nodes))

    


In [1]:
if __name__ == "__main__":
    epsilons = [0.1, 0.8, 0.6, 0.4, 0.2, 0.01]
    for e in epsilons:
        plot_protocol(100,100,steps=5,e=e)
    plt.legend(loc='best')
    plt.show()
    max_delays = [2,4,6,8,10]
    for max_delay in max_delays:
        plot_protocol(100,100,steps=5,max_delay=max_delay)
    plt.legend(loc='best')
    plt.show()

NameError: name 'plot_protocol' is not defined